In [2]:
import pandas as pd
import numpy as np

# Passenger Ordered List

In [3]:
flight_details= pd.read_csv("PNRB-ZZ-20231208_062017.csv")
passenger= pd.read_csv("PNRP-ZZ-20231208_111136.csv")
schedule=pd.read_csv("SCH-ZZ-20231208_035117.csv")
# flight_details

In [4]:
#input

specific_schedule_id= "SCH-ZZ-2098565"
specific_date="20240510"

In [5]:
# segment score and pax score input

external_multiplier_seg = 100
external_multiplier_pax = 50

In [6]:
#ssr cost input and loyalty cost

tags_cost ={'INFT' : 200, 'WCHR' : 200, 'WCHS' : 200, 'WCHC' : 200, 'LANG' : 200, 'CHLD' : 200, 'MAAS' : 200, 'UNMR' : 200, 'BLND' : 200, 'DEAF' : 200, 'EXST' : 200, 'MEAL' : 200, 'NSST' : 200, 'NRPS' : 200,  }
loyalty_cost = {'Platinum' : 2000, 'Gold' : 2000, 'Silver' : 2000, 'none':0 }

In [7]:
tag_columns = ['SPECIAL_NAME_CD2', 'SSR_CODE_CD1']

In [8]:
#generating departure key from input

schedule['FlightNumber']=schedule['FlightNumber'].astype(int)
selected_rows = schedule[schedule['ScheduleID'] == specific_schedule_id][['ScheduleID', 'CarrierCode', 'ArrivalAirport', 'DepartureAirport', 'FlightNumber']]
selected_rows['FlightNumber'] = selected_rows['FlightNumber'].astype(str)
selected_rows['Concatenated'] = selected_rows[['DepartureAirport', 'ArrivalAirport', 'FlightNumber']].astype(str).apply(lambda x: ''.join(x), axis=1)
selected_rows['Concatenated'] = specific_date+selected_rows['Concatenated'].replace(' ', '', regex=True)
selected_rows['Concatenated'] = selected_rows[['CarrierCode','Concatenated']].astype(str).apply(lambda x: ''.join(x), axis=1)

#specific departure key is generated and stored in variable specific_dep_key
specific_dep_key= selected_rows.at[selected_rows.index[0], "Concatenated"]
flight_details["DEP_KEY"]=flight_details["DEP_KEY"].astype(str)
#specific_dep_key+='.0'
specific_dep_key

'ZZ20240510IDRHYD2792'

In [9]:
filtered_f= flight_details[flight_details["DEP_KEY"]== specific_dep_key]
filtered_f_copy= filtered_f
# filtered_f_copy

In [10]:
filtered_f_copy['SEG_TOTAL_MULTIPLIED'] = filtered_f_copy['SEG_TOTAL'] * external_multiplier_seg
filtered_f_copy['PAX_CNT_MULTIPLIED'] = filtered_f_copy['PAX_CNT'] * external_multiplier_pax

# Create a new column with the sum of SEG_SEQ_MULTIPLIED and PAX_CNT_MULTIPLIED
filtered_f_copy['SUM_SEG_PAX'] = filtered_f_copy['SEG_TOTAL_MULTIPLIED'] + filtered_f_copy['PAX_CNT_MULTIPLIED']

C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_26960\4089213020.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_f_copy['SEG_TOTAL_MULTIPLIED'] = filtered_f_copy['SEG_TOTAL'] * external_multiplier_seg
C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_26960\4089213020.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_f_copy['PAX_CNT_MULTIPLIED'] = filtered_f_copy['PAX_CNT'] * external_multiplier_pax
C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_26960\4089213020.py:5: Setti

In [11]:
reclocs_list = filtered_f["RECLOC"].tolist()
filtered_passengers_list = passenger[passenger["RECLOC"].isin(reclocs_list)]
filtered_passengers_list['ssr_score'] = filtered_passengers_list[tag_columns].applymap(tags_cost.get).sum(axis=1)
filtered_passengers_list['Loyalty_score'] = filtered_passengers_list['TierLevel'].map(loyalty_cost).fillna(loyalty_cost['none'])

filtered_passengers_list['Total_score'] = filtered_passengers_list['Loyalty_score'] + filtered_passengers_list['ssr_score']
sorted_passengers=filtered_passengers_list.sort_values(by="Total_score", ascending=False)

# filtered_passengers_list
# sorted_passengers

C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_26960\3454662936.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  filtered_passengers_list['ssr_score'] = filtered_passengers_list[tag_columns].applymap(tags_cost.get).sum(axis=1)
C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_26960\3454662936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_passengers_list['ssr_score'] = filtered_passengers_list[tag_columns].applymap(tags_cost.get).sum(axis=1)
C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_26960\3454662936.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [12]:
merged_sorted_passengers = sorted_passengers.groupby('RECLOC')['Total_score'].sum().reset_index()
# merged_sorted_passengers
merged_table = pd.merge(merged_sorted_passengers, filtered_f_copy, on='RECLOC', how='outer')
merged_table['final_score'] = merged_table['Total_score'] + merged_table['SUM_SEG_PAX']
merged_table=merged_table.drop(['Total_score', 'SEG_TOTAL_MULTIPLIED', 'PAX_CNT_MULTIPLIED' ,'SUM_SEG_PAX'], axis=1)
merge_sorted_table = merged_table.sort_values('final_score', ascending=False)

merge_sorted_table

,RECLOC,CREATION_DTZ,DEP_KEY,ACTION_CD,COS_CD,SEG_SEQ,SEG_TOTAL,PAX_CNT,CARRIER_CD,FLT_NUM,ORIG_CD,DEST_CD,DEP_DT,DEP_DTML,ARR_DTML,DEP_DTMZ,ARR_DTMZ,final_score
0,BUHV67,02-01-2024 08:26,ZZ20240510IDRHYD2792,XX,EconomyClass,1,1,9,ZZ,2792,IDR,HYD,05-10-2024,05-10-2024 12:22,05-10-2024 14:12,05-10-2024 06:52,05-10-2024 08:42,12950.0
9,RHDO70,02-01-2024 08:26,ZZ20240510IDRHYD2792,XX,EconomyClass,1,3,6,ZZ,2792,IDR,HYD,05-10-2024,05-10-2024 12:22,05-10-2024 14:12,05-10-2024 06:52,05-10-2024 08:42,8800.0
6,PJSC24,02-01-2024 08:26,ZZ20240510IDRHYD2792,XX,PremiumEconomyClass,1,3,8,ZZ,2792,IDR,HYD,05-10-2024,05-10-2024 12:22,05-10-2024 14:12,05-10-2024 06:52,05-10-2024 08:42,8700.0
7,PSGX74,02-01-2024 08:26,ZZ20240510IDRHYD2792,XX,BusinessClass,1,4,4,ZZ,2792,IDR,HYD,05-10-2024,05-10-2024 12:22,05-10-2024 14:12,05-10-2024 06:52,05-10-2024 08:42,8600.0
5,MCNQ94,02-01-2024 08:26,ZZ20240510IDRHYD2792,XX,EconomyClass,1,4,4,ZZ,2792,IDR,HYD,05-10-2024,05-10-2024 12:22,05-10-2024 14:12,05-10-2024 06:52,05-10-2024 08:42,7000.0
1,ERZL70,02-01-2024 08:26,ZZ20240510IDRHYD2792,XX,BusinessClass,1,3,4,ZZ,2792,IDR,HYD,05-10-2024,05-10-2024 12:22,05-10-2024 14:12,05-10-2024 06:52,05-10-2024 08:42,4500.0
4,JISG46,02-01-2024 08:26,ZZ20240510IDRHYD2792,XX,PremiumEconomyClass,1,2,2,ZZ,2792,IDR,HYD,05-10-2024,05-10-2024 12:22,05-10-2024 14:12,05-10-2024 06:52,05-10-2024 08:42,4300.0
8,QXQZ78,02-01-2024 08:26,ZZ20240510IDRHYD2792,XX,PremiumEconomyClass,1,1,4,ZZ,2792,IDR,HYD,05-10-2024,05-10-2024 12:22,05-10-2024 14:12,05-10-2024 06:52,05-10-2024 08:42,4300.0
2,GQDX67,02-01-2024 08:26,ZZ20240510IDRHYD2792,XX,FirstClass,1,2,3,ZZ,2792,IDR,HYD,05-10-2024,05-10-2024 12:22,05-10-2024 14:12,05-10-2024 06:52,05-10-2024 08:42,2550.0
10,SQMP46,02-01-2024 08:26,ZZ20240510IDRHYD2792,XX,FirstClass,1,4,2,ZZ,2792,IDR,HYD,05-10-2024,05-10-2024 12:22,05-10-2024 14:12,05-10-2024 06:52,05-10-2024 08:42,2500.0


In [13]:
final_recloc_list = merge_sorted_table['RECLOC'].tolist()
final_pax_cnt=merge_sorted_table['PAX_CNT'].tolist()
final_seg_seq_cnt=merge_sorted_table['SEG_SEQ'].tolist()
final_score=merge_sorted_table['final_score'].tolist()



len(final_pax_cnt)

15

# Direct Flight Ordered List

In [14]:
specific_dep_key

'ZZ20240510IDRHYD2792'

In [15]:
f = pd.read_csv('INV-ZZ-20231208_041852.csv')
f['DepartureDateTime'] = pd.to_datetime(f['DepartureDateTime'])
f['ArrivalDateTime'] = pd.to_datetime(f['ArrivalDateTime'])

In [16]:
def flights_between(departure_airport,arrival_airport,flight_schedule_df):
    filtered_flights = flight_schedule_df[
        (flight_schedule_df['DepartureAirport'] == departure_airport) &
        (flight_schedule_df['ArrivalAirport'] == arrival_airport)
    ]
    return filtered_flights

In [17]:
# Function to get City Mapping like Key(Arrival City): Values(List of Departure Cities)
city_mapping = {}

# Iterate over each row in the DataFrame
for index, row in f.iterrows():
    arrival_city = row['ArrivalAirport']
    departure_city = row['DepartureAirport']

    # Check if arrival city is already in the dictionary
    if arrival_city in city_mapping:
        # Append departure city to the existing list
        city_mapping[arrival_city].append(departure_city)
    else:
        # Create a new list for the arrival city
        city_mapping[arrival_city] = [departure_city]
for key in city_mapping.keys():
    city_mapping[key] = np.unique(city_mapping[key])
city_mapping

{'HYD': array(['AMD', 'BLR', 'BOM', 'CCU', 'CJB', 'IDR', 'MAA'], dtype='<U3'),
 'AMD': array(['BLR', 'BOM', 'DEL', 'HYD', 'MAA'], dtype='<U3'),
 'MAA': array(['AMD', 'BOM', 'CCU', 'DEL', 'GOI'], dtype='<U3'),
 'CCU': array(['AMD', 'BLR', 'BOM', 'HYD', 'MAA'], dtype='<U3'),
 'BOM': array(['AMD', 'BLR', 'CCU', 'DEL', 'GAU', 'GOI', 'MAA', 'PNQ'],
       dtype='<U3'),
 'BLR': array(['AMD', 'BBI', 'BOM', 'CCU', 'DEL', 'HYD', 'JAI', 'LKO'],
       dtype='<U3'),
 'DEL': array(['AMD', 'BLR', 'BOM', 'HYD', 'JAI'], dtype='<U3'),
 'CJB': array(['HYD', 'MAA'], dtype='<U3'),
 'TRV': array(['MAA'], dtype='<U3'),
 'PNQ': array(['BOM', 'GAU', 'GOI'], dtype='<U3'),
 'IDR': array(['HYD'], dtype='<U3'),
 'LKO': array(['JAI'], dtype='<U3'),
 'GAU': array(['GOI', 'PNQ'], dtype='<U3'),
 'GOI': array(['PNQ'], dtype='<U3')}

In [18]:
b = flights_between(city_mapping['BLR'][0],'BLR',f)
b

,InventoryId,ScheduleId,CarrierCode,Dep_Key,FlightNumber,AircraftType,DepartureDate,DepartureDateTime,ArrivalDateTime,DepartureAirport,...,PC_Oversold,PC_AvailableInventory,EC_TotalInventory,EC_BookedInventory,EC_Oversold,EC_AvailableInventory,FC_CD,BC_CD,PC_CD,EC_CD
892,INV-ZZ-7703162,SCH-ZZ-5156445,ZZ,ZZ20240514AMDBLR3782.0,3782.0,Boeing 787,05/14/2024,2024-05-14 15:57:00,2024-05-15 07:24:00,AMD,...,32,43,120,65,43,55,"{'F': 16, 'P': 11}","{'C': 22, 'J': 16, 'Z': 16}","{'Q': 24, 'R': 16, 'S': 8, 'T': 8, 'H': 16, 'M...","{'Y': 8, 'A': 8, 'B': 4, 'D': 4, 'E': 4, 'G': ..."
893,INV-ZZ-7431689,SCH-ZZ-5156445,ZZ,ZZ20240515AMDBLR3782.0,3782.0,Boeing 787,05/15/2024,2024-05-15 15:57:00,2024-05-16 07:24:00,AMD,...,33,42,125,66,42,59,"{'F': 16, 'P': 11}","{'C': 22, 'J': 16, 'Z': 16}","{'Q': 24, 'R': 16, 'S': 8, 'T': 8, 'H': 16, 'M...","{'Y': 8, 'A': 8, 'B': 4, 'D': 4, 'E': 4, 'G': ..."
894,INV-ZZ-1603837,SCH-ZZ-5156445,ZZ,ZZ20240517AMDBLR3782.0,3782.0,Boeing 787,05/17/2024,2024-05-17 15:57:00,2024-05-18 07:24:00,AMD,...,35,50,119,57,51,62,"{'F': 16, 'P': 11}","{'C': 22, 'J': 16, 'Z': 16}","{'Q': 24, 'R': 16, 'S': 8, 'T': 8, 'H': 16, 'M...","{'Y': 8, 'A': 8, 'B': 4, 'D': 4, 'E': 4, 'G': ..."
895,INV-ZZ-6060194,SCH-ZZ-5156445,ZZ,ZZ20240518AMDBLR3782.0,3782.0,Boeing 787,05/18/2024,2024-05-18 15:57:00,2024-05-19 07:24:00,AMD,...,35,49,121,58,50,63,"{'F': 16, 'P': 11}","{'C': 22, 'J': 16, 'Z': 16}","{'Q': 24, 'R': 16, 'S': 8, 'T': 8, 'H': 16, 'M...","{'Y': 8, 'A': 8, 'B': 4, 'D': 4, 'E': 4, 'G': ..."
896,INV-ZZ-3258972,SCH-ZZ-5156445,ZZ,ZZ20240521AMDBLR3782.0,3782.0,Boeing 787,05/21/2024,2024-05-21 15:57:00,2024-05-22 07:24:00,AMD,...,35,45,123,62,46,61,"{'F': 16, 'P': 11}","{'C': 22, 'J': 16, 'Z': 16}","{'Q': 24, 'R': 16, 'S': 8, 'T': 8, 'H': 16, 'M...","{'Y': 8, 'A': 8, 'B': 4, 'D': 4, 'E': 4, 'G': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198,INV-ZZ-4855587,SCH-ZZ-4843596,ZZ,ZZ20240519AMDBLR3041.0,3041.0,Airbus A321-200,05/19/2024,2024-05-19 14:26:00,2024-05-19 18:12:00,AMD,...,28,36,93,44,36,49,"{'F': 12, 'P': 8}","{'C': 16, 'J': 12, 'Z': 12}","{'Q': 18, 'R': 12, 'S': 6, 'T': 6, 'H': 12, 'M...","{'Y': 6, 'A': 6, 'B': 3, 'D': 3, 'E': 3, 'G': ..."
1199,INV-ZZ-4725843,SCH-ZZ-4843596,ZZ,ZZ20240520AMDBLR3041.0,3041.0,Airbus A321-200,05/20/2024,2024-05-20 14:26:00,2024-05-20 18:12:00,AMD,...,27,36,92,45,35,47,"{'F': 12, 'P': 8}","{'C': 16, 'J': 12, 'Z': 12}","{'Q': 18, 'R': 12, 'S': 6, 'T': 6, 'H': 12, 'M...","{'Y': 6, 'A': 6, 'B': 3, 'D': 3, 'E': 3, 'G': ..."
1200,INV-ZZ-1805422,SCH-ZZ-4843596,ZZ,ZZ20240524AMDBLR3041.0,3041.0,Airbus A321-200,05/24/2024,2024-05-24 14:26:00,2024-05-24 18:12:00,AMD,...,26,33,92,46,34,46,"{'F': 12, 'P': 8}","{'C': 16, 'J': 12, 'Z': 12}","{'Q': 18, 'R': 12, 'S': 6, 'T': 6, 'H': 12, 'M...","{'Y': 6, 'A': 6, 'B': 3, 'D': 3, 'E': 3, 'G': ..."
1201,INV-ZZ-6244039,SCH-ZZ-4843596,ZZ,ZZ20240526AMDBLR3041.0,3041.0,Airbus A321-200,05/26/2024,2024-05-26 14:26:00,2024-05-26 18:12:00,AMD,...,29,36,90,41,39,49,"{'F': 12, 'P': 8}","{'C': 16, 'J': 12, 'Z': 12}","{'Q': 18, 'R': 12, 'S': 6, 'T': 6, 'H': 12, 'M...","{'Y': 6, 'A': 6, 'B': 3, 'D': 3, 'E': 3, 'G': ..."


In [19]:
print(type(b.iloc[3]))

<class 'pandas.core.series.Series'>


In [20]:
import pandas as pd

def arrival_delay_cost_function(time_diff):
    if(time_diff <= pd.Timedelta('6 hours')):
        return 70
    elif(time_diff <= pd.Timedelta('12 hours')):
        return 50
    elif(time_diff <= pd.Timedelta('24 hours')):
        return 40
    elif(time_diff <= pd.Timedelta('48 hours')):
        return 30
    else:
        return 0
def departure_delay_cost_function(time_diff):
    if(time_diff <= pd.Timedelta('6 hours')):
        return 70
    elif(time_diff <= pd.Timedelta('12 hours')):
        return 50
    elif(time_diff <= pd.Timedelta('24 hours')):
        return 40
    elif(time_diff <= pd.Timedelta('48 hours')):
        return 30
    else:
        return 0
        
def alternate_direct_flights(cancelled_flight_dep_key, df):
    cancelled_flight = df[df['Dep_Key'] == cancelled_flight_dep_key+'.0'].iloc[0]
    flights_list = flights_between(cancelled_flight['DepartureAirport'], cancelled_flight['ArrivalAirport'], df).copy()
    
    flights_list['DepartureDateTimeDiff'] = [i-cancelled_flight['DepartureDateTime'] for i in flights_list['DepartureDateTime']]
    flights_list['ArrivalDateTimeDiff'] = [i-cancelled_flight['ArrivalDateTime'] for i in flights_list['ArrivalDateTime']]

        
    flights_list_ = flights_list.copy().iloc[0:0]
    for i in range(len(flights_list)):
        if(((flights_list.iloc[i].DepartureDateTimeDiff) >= pd.Timedelta('0 days')) and ((flights_list.iloc[i].ArrivalDateTimeDiff) >= pd.Timedelta('0 days'))):
            if(((flights_list.iloc[i].DepartureDateTimeDiff) <= pd.Timedelta('2 days')) and ((flights_list.iloc[i].ArrivalDateTimeDiff) <= pd.Timedelta('2 days'))):
                flights_list_.loc[len(flights_list_)]=flights_list.iloc[i]

    flights_list_['Cost'] = 0
    for i in range(len(flights_list_)):
        flights_list_['Cost'].iloc[i] += arrival_delay_cost_function(flights_list_['ArrivalDateTimeDiff'].iloc[i])
        flights_list_['Cost'].iloc[i] += departure_delay_cost_function(flights_list_['DepartureDateTimeDiff'].iloc[i])
        flights_list_['Cost'].iloc[i] += 40 # Same City Pairs
    return flights_list_.sort_values(by='Cost', ascending=False).iloc[1:]

# Assuming y and f are defined somewhere
x = alternate_direct_flights(specific_dep_key, f)
x

C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_26960\1152031819.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights_list_['Cost'].iloc[i] += arrival_delay_cost_function(flights_list_['ArrivalDateTimeDiff'].iloc[i])
C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_26960\1152031819.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights_list_['Cost'].iloc[i] += departure_delay_cost_function(flights_list_['DepartureDateTimeDiff'].iloc[i])
C:\Users\Vamsi Krishna\AppData\Local\Temp\ipykernel_26960\1152031819.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

,InventoryId,ScheduleId,CarrierCode,Dep_Key,FlightNumber,AircraftType,DepartureDate,DepartureDateTime,ArrivalDateTime,DepartureAirport,...,EC_BookedInventory,EC_Oversold,EC_AvailableInventory,FC_CD,BC_CD,PC_CD,EC_CD,DepartureDateTimeDiff,ArrivalDateTimeDiff,Cost
2,INV-ZZ-7767976,SCH-ZZ-6755803,ZZ,ZZ20240511IDRHYD3455.0,3455.0,Boeing 777,05/11/2024,2024-05-11 03:41:00,2024-05-12 01:59:00,IDR,...,107,63,84,"{'F': 25, 'P': 17}","{'C': 34, 'J': 26, 'Z': 26}","{'Q': 38, 'R': 25, 'S': 13, 'T': 13, 'H': 25, ...","{'Y': 13, 'A': 13, 'B': 6, 'D': 6, 'E': 6, 'G'...",0 days 15:19:00,1 days 11:47:00,110
1,INV-ZZ-5004957,SCH-ZZ-2098565,ZZ,ZZ20240512IDRHYD2792.0,2792.0,BAe 146,05/12/2024,2024-05-12 12:22:00,2024-05-12 14:12:00,IDR,...,20,16,20,"{'F': 5, 'P': 4}","{'C': 7, 'J': 5, 'Z': 5}","{'Q': 8, 'R': 5, 'S': 3, 'T': 3, 'H': 5, 'M': 3}","{'Y': 3, 'A': 3, 'B': 1, 'D': 1, 'E': 1, 'G': ...",2 days 00:00:00,2 days 00:00:00,100


In [21]:
(x.AvailableInventory).tolist()

[209, 52]

In [22]:
(x.Cost)

2    110
1    100
Name: Cost, dtype: int64

In [23]:
a=(x.AvailableInventory).tolist()
ac=(x.Cost).tolist()
pc=final_score
p=final_pax_cnt

m=len(a)
n=len(p)

p


[9, 6, 8, 4, 4, 4, 2, 4, 3, 2, 1, 1, 1, 1, 2]

In [24]:
pas_in_fli = [[f'pas_{i}_in_fli_{j}' for j in range(m)] for i in range(n)]
pas_in_fli

[['pas_0_in_fli_0', 'pas_0_in_fli_1'],
 ['pas_1_in_fli_0', 'pas_1_in_fli_1'],
 ['pas_2_in_fli_0', 'pas_2_in_fli_1'],
 ['pas_3_in_fli_0', 'pas_3_in_fli_1'],
 ['pas_4_in_fli_0', 'pas_4_in_fli_1'],
 ['pas_5_in_fli_0', 'pas_5_in_fli_1'],
 ['pas_6_in_fli_0', 'pas_6_in_fli_1'],
 ['pas_7_in_fli_0', 'pas_7_in_fli_1'],
 ['pas_8_in_fli_0', 'pas_8_in_fli_1'],
 ['pas_9_in_fli_0', 'pas_9_in_fli_1'],
 ['pas_10_in_fli_0', 'pas_10_in_fli_1'],
 ['pas_11_in_fli_0', 'pas_11_in_fli_1'],
 ['pas_12_in_fli_0', 'pas_12_in_fli_1'],
 ['pas_13_in_fli_0', 'pas_13_in_fli_1'],
 ['pas_14_in_fli_0', 'pas_14_in_fli_1']]

In [25]:
coe=[]

for k in range(m*n):
    coe.append(-(pc[k//m]+ac[k%m]))

coe

[-13060.0,
 -13050.0,
 -8910.0,
 -8900.0,
 -8810.0,
 -8800.0,
 -8710.0,
 -8700.0,
 -7110.0,
 -7100.0,
 -4610.0,
 -4600.0,
 -4410.0,
 -4400.0,
 -4410.0,
 -4400.0,
 -2660.0,
 -2650.0,
 -2610.0,
 -2600.0,
 -2560.0,
 -2550.0,
 -2560.0,
 -2550.0,
 -2360.0,
 -2350.0,
 -460.0,
 -450.0,
 -410.0,
 -400.0]

In [26]:
list=[]
for k in pas_in_fli:
    for n in k:
        list.append(n)

linear=dict(zip(list,coe))
linear

{'pas_0_in_fli_0': -13060.0,
 'pas_0_in_fli_1': -13050.0,
 'pas_1_in_fli_0': -8910.0,
 'pas_1_in_fli_1': -8900.0,
 'pas_2_in_fli_0': -8810.0,
 'pas_2_in_fli_1': -8800.0,
 'pas_3_in_fli_0': -8710.0,
 'pas_3_in_fli_1': -8700.0,
 'pas_4_in_fli_0': -7110.0,
 'pas_4_in_fli_1': -7100.0,
 'pas_5_in_fli_0': -4610.0,
 'pas_5_in_fli_1': -4600.0,
 'pas_6_in_fli_0': -4410.0,
 'pas_6_in_fli_1': -4400.0,
 'pas_7_in_fli_0': -4410.0,
 'pas_7_in_fli_1': -4400.0,
 'pas_8_in_fli_0': -2660.0,
 'pas_8_in_fli_1': -2650.0,
 'pas_9_in_fli_0': -2610.0,
 'pas_9_in_fli_1': -2600.0,
 'pas_10_in_fli_0': -2560.0,
 'pas_10_in_fli_1': -2550.0,
 'pas_11_in_fli_0': -2560.0,
 'pas_11_in_fli_1': -2550.0,
 'pas_12_in_fli_0': -2360.0,
 'pas_12_in_fli_1': -2350.0,
 'pas_13_in_fli_0': -460.0,
 'pas_13_in_fli_1': -450.0,
 'pas_14_in_fli_0': -410.0,
 'pas_14_in_fli_1': -400.0}

In [27]:
from dimod import BinaryQuadraticModel
import hybrid
from hybrid import SimulatedAnnealingProblemSampler,TabuProblemSampler,min_sample,State

bqm = BinaryQuadraticModel(linear,{},"BINARY")
lmbd = 10000

In [28]:
for i in range(len(p)):
    cons1=[]
    for j in range(len(ac)):
        cons1.append((f'pas_{i}_in_fli_{j}', 1))
    print(cons1)
    bqm.add_linear_equality_constraint(cons1, lmbd*10, -1)

# for each passenger get 1 flight

[('pas_0_in_fli_0', 1), ('pas_0_in_fli_1', 1)]
[('pas_1_in_fli_0', 1), ('pas_1_in_fli_1', 1)]
[('pas_2_in_fli_0', 1), ('pas_2_in_fli_1', 1)]
[('pas_3_in_fli_0', 1), ('pas_3_in_fli_1', 1)]
[('pas_4_in_fli_0', 1), ('pas_4_in_fli_1', 1)]
[('pas_5_in_fli_0', 1), ('pas_5_in_fli_1', 1)]
[('pas_6_in_fli_0', 1), ('pas_6_in_fli_1', 1)]
[('pas_7_in_fli_0', 1), ('pas_7_in_fli_1', 1)]
[('pas_8_in_fli_0', 1), ('pas_8_in_fli_1', 1)]
[('pas_9_in_fli_0', 1), ('pas_9_in_fli_1', 1)]
[('pas_10_in_fli_0', 1), ('pas_10_in_fli_1', 1)]
[('pas_11_in_fli_0', 1), ('pas_11_in_fli_1', 1)]
[('pas_12_in_fli_0', 1), ('pas_12_in_fli_1', 1)]
[('pas_13_in_fli_0', 1), ('pas_13_in_fli_1', 1)]
[('pas_14_in_fli_0', 1), ('pas_14_in_fli_1', 1)]


In [29]:
for j in range(len(a)):
    cons2=[]
    
    for i in range(len(p)):
        cons2.append((f'pas_{i}_in_fli_{j}',p[i]))
    print(cons2,a[j])   
    bqm.add_linear_inequality_constraint(terms=cons2, lagrange_multiplier=lmbd*10, constant=0, label='s', lb=0, ub=a[j])
    

[('pas_0_in_fli_0', 9), ('pas_1_in_fli_0', 6), ('pas_2_in_fli_0', 8), ('pas_3_in_fli_0', 4), ('pas_4_in_fli_0', 4), ('pas_5_in_fli_0', 4), ('pas_6_in_fli_0', 2), ('pas_7_in_fli_0', 4), ('pas_8_in_fli_0', 3), ('pas_9_in_fli_0', 2), ('pas_10_in_fli_0', 1), ('pas_11_in_fli_0', 1), ('pas_12_in_fli_0', 1), ('pas_13_in_fli_0', 1), ('pas_14_in_fli_0', 2)] 209
[('pas_0_in_fli_1', 9), ('pas_1_in_fli_1', 6), ('pas_2_in_fli_1', 8), ('pas_3_in_fli_1', 4), ('pas_4_in_fli_1', 4), ('pas_5_in_fli_1', 4), ('pas_6_in_fli_1', 2), ('pas_7_in_fli_1', 4), ('pas_8_in_fli_1', 3), ('pas_9_in_fli_1', 2), ('pas_10_in_fli_1', 1), ('pas_11_in_fli_1', 1), ('pas_12_in_fli_1', 1), ('pas_13_in_fli_1', 1), ('pas_14_in_fli_1', 2)] 52


c:\Users\Vamsi Krishna\anaconda3\envs\qc\lib\site-packages\dimod\binary\binary_quadratic_model.py:770: UserWarning: Did not add constraint s. This constraint is feasible with any value for state variables.
  warnings.warn(


In [30]:
sampler = SimulatedAnnealingProblemSampler()
state = State.from_sample(min_sample(bqm), bqm)
new_state = sampler.run(state).result()
solute=new_state.samples.first.sample, new_state.samples.first.energy

In [31]:
solute[0]

{'pas_0_in_fli_0': 1,
 'pas_0_in_fli_1': 0,
 'pas_10_in_fli_0': 1,
 'pas_10_in_fli_1': 0,
 'pas_11_in_fli_0': 1,
 'pas_11_in_fli_1': 0,
 'pas_12_in_fli_0': 0,
 'pas_12_in_fli_1': 1,
 'pas_13_in_fli_0': 0,
 'pas_13_in_fli_1': 1,
 'pas_14_in_fli_0': 1,
 'pas_14_in_fli_1': 0,
 'pas_1_in_fli_0': 0,
 'pas_1_in_fli_1': 1,
 'pas_2_in_fli_0': 1,
 'pas_2_in_fli_1': 0,
 'pas_3_in_fli_0': 1,
 'pas_3_in_fli_1': 0,
 'pas_4_in_fli_0': 1,
 'pas_4_in_fli_1': 0,
 'pas_5_in_fli_0': 0,
 'pas_5_in_fli_1': 1,
 'pas_6_in_fli_0': 1,
 'pas_6_in_fli_1': 0,
 'pas_7_in_fli_0': 1,
 'pas_7_in_fli_1': 0,
 'pas_8_in_fli_0': 0,
 'pas_8_in_fli_1': 1,
 'pas_9_in_fli_0': 0,
 'pas_9_in_fli_1': 1}

In [32]:
no_pas=0

for i in range(len(p)):
    for j in range(len(a)):
     if solute[0][f'pas_{i}_in_fli_{j}']==1:
        no_pas=no_pas+1
no_pas

15

In [33]:
fli_cap=[]
print(a)
for j in range(len(a)):
    count=0
    for i in range(len(pc)):
        if solute[0][f'pas_{i}_in_fli_{j}']==1:
            count=count+p[i]
    fli_cap.append(count) 

fli_cap

[209, 52]


[35, 17]

In [34]:
for i in range(len(p)):
 sum=0
 for j in range(len(a)):
  sum=sum+solute[0][f'pas_{i}_in_fli_{j}']
  if sum >1 :
    solute[0][f'pas_{i}_in_fli_{j}']=0


In [35]:
total_available_updated = []
for i in range(len(a)):
    total_available_updated.append(a[i]-fli_cap[i])
print(a)
total_available_updated

[209, 52]


[174, 35]

In [36]:
og_booked_inv = x.BookedInventory.tolist()
updated_booked_inv = []
for i in range(len(og_booked_inv)):
    updated_booked_inv.append(og_booked_inv[i]+fli_cap[i])
print(og_booked_inv)
updated_booked_inv

[268, 54]


[303, 71]

In [37]:
depkey = x.Dep_Key.tolist()
depkey


['ZZ20240511IDRHYD3455.0', 'ZZ20240512IDRHYD2792.0']

In [38]:
import pandas as pd

file_path = r"C:\Users\Vamsi Krishna\codes\QC\New folder\Updated_INV.csv"
df = pd.read_csv(file_path)
print(df)

for i in range(len(a)):
    dep_key_to_update = depkey[i]
    new_available_inventory = total_available_updated[i]
    new_booked_inventory = updated_booked_inv[i]

    row_index = df.index[df['Dep_Key'] == dep_key_to_update].tolist()

    if row_index:
        row_index = row_index[0] 
        
        df.at[row_index, 'AvailableInventory'] = new_available_inventory
        df.at[row_index, 'BookedInventory'] = new_booked_inventory

        df.to_csv(file_path, index=False)
        print(f"Values for Dep_Key '{dep_key_to_update}' updated successfully.")
    else:
        print(f"Dep_Key '{dep_key_to_update}' not found in the DataFrame.")


     ZZ20240511IDRHYD3455.0      ScheduleId CarrierCode  \
0            INV-ZZ-2774494  SCH-ZZ-3400127          ZZ   
1            INV-ZZ-8029879  SCH-ZZ-3400127          ZZ   
2            INV-ZZ-6852033  SCH-ZZ-3400127          ZZ   
3            INV-ZZ-7065859  SCH-ZZ-3400127          ZZ   
4            INV-ZZ-1595392  SCH-ZZ-3400127          ZZ   
...                     ...             ...         ...   
2007         INV-ZZ-6129434  SCH-ZZ-9762124          ZZ   
2008         INV-ZZ-2755720  SCH-ZZ-9762124          ZZ   
2009         INV-ZZ-2707415  SCH-ZZ-9762124          ZZ   
2010         INV-ZZ-1293533  SCH-ZZ-9762124          ZZ   
2011         INV-ZZ-1560576  SCH-ZZ-9762124          ZZ   

                     Dep_Key  FlightNumber AircraftType DepartureDate  \
0     ZZ20240505AMDHYD2223.0          2223   Boeing 777    05-05-2024   
1     ZZ20240506AMDHYD2223.0          2223   Boeing 777    05-06-2024   
2     ZZ20240507AMDHYD2223.0          2223   Boeing 777    05-07-2024   

In [39]:
print(final_recloc_list)
print(solute[0])

['BUHV67', 'RHDO70', 'PJSC24', 'PSGX74', 'MCNQ94', 'ERZL70', 'JISG46', 'QXQZ78', 'GQDX67', 'SQMP46', 'HQAO84', 'ZTQP69', 'WYWE97', 'TWFD73', 'YXHR85']
{'pas_0_in_fli_0': 1, 'pas_0_in_fli_1': 0, 'pas_10_in_fli_0': 1, 'pas_10_in_fli_1': 0, 'pas_11_in_fli_0': 1, 'pas_11_in_fli_1': 0, 'pas_12_in_fli_0': 0, 'pas_12_in_fli_1': 1, 'pas_13_in_fli_0': 0, 'pas_13_in_fli_1': 1, 'pas_14_in_fli_0': 1, 'pas_14_in_fli_1': 0, 'pas_1_in_fli_0': 0, 'pas_1_in_fli_1': 1, 'pas_2_in_fli_0': 1, 'pas_2_in_fli_1': 0, 'pas_3_in_fli_0': 1, 'pas_3_in_fli_1': 0, 'pas_4_in_fli_0': 1, 'pas_4_in_fli_1': 0, 'pas_5_in_fli_0': 0, 'pas_5_in_fli_1': 1, 'pas_6_in_fli_0': 1, 'pas_6_in_fli_1': 0, 'pas_7_in_fli_0': 1, 'pas_7_in_fli_1': 0, 'pas_8_in_fli_0': 0, 'pas_8_in_fli_1': 1, 'pas_9_in_fli_0': 0, 'pas_9_in_fli_1': 1}


In [40]:
depkey

['ZZ20240511IDRHYD3455.0', 'ZZ20240512IDRHYD2792.0']

In [41]:
import pandas as pd
import re
from datetime import datetime, timedelta

pnr_df = pd.read_csv(r"C:\Users\Vamsi Krishna\codes\QC\New folder\Updated_PNR.csv")

pnr_df = pnr_df.drop(0)

for key, value in solute[0].items():
    if value == 1:
        pas_match = re.search(r'\d+', key)
        pas_index = int(pas_match.group()) if pas_match else None
        fli_index = int(key.split('_in_fli_')[1])

        if pas_index is not None:
            depkey_info = depkey[fli_index]
            carrier_code = depkey_info[:2]
            dep_dt = datetime.strptime(depkey_info[2:10], '%Y%m%d').strftime('%m-%d-%Y')
            
            flight_num_match = re.search(r'\d+', depkey_info[10:])
            flight_num = int(flight_num_match.group()) if flight_num_match else None
            
            orig_cd = depkey_info[10:13]
            dest_cd = depkey_info[13:16]
            upd_depkey = depkey_info.split('.')[0]

            dep_time = x.loc[x['Dep_Key'] == depkey_info, 'DepartureDateTime'].values[0]
            arr_time = x.loc[x['Dep_Key'] == depkey_info, 'ArrivalDateTime'].values[0]

            recloc_col_index = pnr_df.columns.get_loc('RECLOC')
            recloc = final_recloc_list[pas_index]
            pnr_df.loc[pnr_df.iloc[:, recloc_col_index] == recloc, 'DEP_KEY'] = upd_depkey
            pnr_df.loc[pnr_df.iloc[:, recloc_col_index] == recloc, 'CARRIER_CD'] = carrier_code
            pnr_df.loc[pnr_df.iloc[:, recloc_col_index] == recloc, 'FLT_NUM'] = flight_num
            pnr_df.loc[pnr_df.iloc[:, recloc_col_index] == recloc, 'DEP_DT'] = dep_dt
            pnr_df.loc[pnr_df.iloc[:, recloc_col_index] == recloc, 'ORIG_CD'] = orig_cd
            pnr_df.loc[pnr_df.iloc[:, recloc_col_index] == recloc, 'DEST_CD'] = dest_cd
            pnr_df.loc[pnr_df.iloc[:, recloc_col_index] == recloc, 'DEP_DTML'] = dep_time
            pnr_df.loc[pnr_df.iloc[:, recloc_col_index] == recloc, 'ARR_DTML'] = arr_time
            pnr_df.loc[pnr_df.iloc[:, recloc_col_index] == recloc, 'DEP_DTMZ'] = (dep_time - timedelta(hours=5, minutes=30)).strftime('%m-%d-%Y %H:%M:%S')
            pnr_df.loc[pnr_df.iloc[:, recloc_col_index] == recloc, 'ARR_DTMZ'] = (arr_time - timedelta(hours=5, minutes=30)).strftime('%m-%d-%Y %H:%M:%S')

pnr_df.to_csv(r"C:\Users\Vamsi Krishna\codes\QC\New folder\Updated_PNR.csv", index=False)


In [46]:
import pandas as pd

df = pd.read_csv(r"C:\Users\Vamsi Krishna\codes\QC\New folder\1\test_inv.csv")


depkey_without_suffix = [key.replace('.0', '') for key in depkey]

result_df = pd.DataFrame()

for desired_dep_key in depkey_without_suffix:
    result_row = df[df['Dep_Key'].str.contains(desired_dep_key, case=False, na=False)]
    
    if not result_row.empty:
        result_df = pd.concat([result_df, result_row])

result_df['Dep_Key'] = result_df['Dep_Key'].astype(str).str.rstrip('.0')


result_df.to_csv(r"C:\Users\Vamsi Krishna\codes\QC\New folder\1\Sorted_INV.csv", index=False)




In [54]:
import pandas as pd

df = pd.read_csv(r"C:\Users\Vamsi Krishna\codes\QC\New folder\1\test_pnr.csv")


pnr_result_df = pd.DataFrame()

for desired_dep_key in final_recloc_list:
    result_row = df[df['RECLOC'].str.contains(desired_dep_key, case=False, na=False)]
    
    if not result_row.empty:
        pnr_result_df = pd.concat([pnr_result_df, result_row])

pnr_result_df['RECLOC'] = pnr_result_df['RECLOC'].astype(str).str.rstrip('.0')
pnr_result_df = pnr_result_df.drop('COS_CD',axis=1)

pnr_result_df.to_csv(r"C:\Users\Vamsi Krishna\codes\QC\New folder\1\Sorted_PNR.csv", index=False)


In [55]:
result_df

,InventoryId,ScheduleId,CarrierCode,Dep_Key,FlightNumber,AircraftType,DepartureDate,DepartureDateTime,ArrivalDateTime,DepartureAirport,...,PC_Oversold,PC_AvailableInventory,EC_TotalInventory,EC_BookedInventory,EC_Oversold,EC_AvailableInventory,FC_CD,BC_CD,PC_CD,EC_CD
1686,INV-ZZ-7767976,SCH-ZZ-6755803,ZZ,ZZ20240511IDRHYD3455,3455.0,Boeing 777,05/11/2024,2024-05-11 03:41:00,2024-05-12 01:59:00,IDR,...,47,58,191,116,63,75,"{'F': 25, 'P': 17}","{'C': 34, 'J': 26, 'Z': 26}","{'Q': 38, 'R': 25, 'S': 13, 'T': 13, 'H': 25, ...","{'Y': 13, 'A': 13, 'B': 6, 'D': 6, 'E': 6, 'G'..."
1389,INV-ZZ-5004957,SCH-ZZ-2098565,ZZ,ZZ20240512IDRHYD2792,2792.0,BAe 146,05/12/2024,2024-05-12 12:22:00,2024-05-12 14:12:00,IDR,...,11,5,40,32,16,8,"{'F': 5, 'P': 4}","{'C': 7, 'J': 5, 'Z': 5}","{'Q': 8, 'R': 5, 'S': 3, 'T': 3, 'H': 5, 'M': 3}","{'Y': 3, 'A': 3, 'B': 1, 'D': 1, 'E': 1, 'G': ..."


In [56]:
pnr_result_df

,RECLOC,CREATION_DTZ,DEP_KEY,ACTION_CD,SEG_SEQ,SEG_TOTAL,PAX_CNT,CARRIER_CD,FLT_NUM,ORIG_CD,DEST_CD,DEP_DT,DEP_DTML,ARR_DTML,DEP_DTMZ,ARR_DTMZ
32923,BUHV67,2/1/2024 8:26,ZZ20240510IDRHYD2792,XX,1,1,9,ZZ,2792,IDR,HYD,5/10/2024,5/10/2024 12:22,5/10/2024 14:12,5/10/2024 6:52,5/10/2024 8:42
32924,RHDO7,2/1/2024 8:26,ZZ20240510IDRHYD2792,XX,1,3,6,ZZ,2792,IDR,HYD,5/10/2024,5/10/2024 12:22,5/10/2024 14:12,5/10/2024 6:52,5/10/2024 8:42
32940,RHDO7,2/1/2024 8:26,ZZ20240511HYDAMD3723,HL,2,3,6,ZZ,3723,HYD,AMD,5/11/2024,5/11/2024 20:40,5/12/2024 12:24,5/11/2024 15:10,5/12/2024 6:54
32941,RHDO7,2/1/2024 8:26,ZZ20240512AMDBLR3041,HK,3,3,6,ZZ,3041,AMD,BLR,5/12/2024,5/12/2024 14:26,5/12/2024 18:12,5/12/2024 8:56,5/12/2024 12:42
32918,PJSC24,2/1/2024 8:26,ZZ20240511IDRHYD3455,XX,1,3,8,ZZ,3455,IDR,HYD,05-11-2024,2024-05-11 03:41:00,2024-05-12 01:59:00,05-10-2024 22:11:00,05-11-2024 20:29:00
32933,PJSC24,2/1/2024 8:26,ZZ20240513HYDCJB2914,XX,2,3,8,ZZ,2914,HYD,CJB,5/13/2024,5/13/2024 1:16,5/13/2024 9:23,5/12/2024 19:46,5/13/2024 3:53
32914,PSGX74,2/1/2024 8:26,ZZ20240510IDRHYD2792,XX,1,4,4,ZZ,2792,IDR,HYD,5/10/2024,5/10/2024 12:22,5/10/2024 14:12,5/10/2024 6:52,5/10/2024 8:42
32931,PSGX74,2/1/2024 8:26,ZZ20240511HYDCCU3772,HK,2,4,4,ZZ,3772,HYD,CCU,5/11/2024,5/11/2024 5:58,5/12/2024 3:23,5/11/2024 0:28,5/11/2024 21:53
32922,MCNQ94,2/1/2024 8:26,ZZ20240510IDRHYD2792,XX,1,4,4,ZZ,2792,IDR,HYD,5/10/2024,5/10/2024 12:22,5/10/2024 14:12,5/10/2024 6:52,5/10/2024 8:42
32939,MCNQ94,2/1/2024 8:26,ZZ20240511HYDCCU3772,GN,2,4,4,ZZ,3772,HYD,CCU,5/11/2024,5/11/2024 5:58,5/12/2024 3:23,5/11/2024 0:28,5/11/2024 21:53
